In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train=pd.read_excel('Data_Train.xlsx')
test=pd.read_excel('Data_Test.xlsx')

In [ ]:
train.isnull().sum()

Restaurant          0
Location            0
Cuisines            0
Average_Cost        1
Minimum_Order       0
Rating           1963
Votes            2074
Reviews          2312
Delivery_Time       0
dtype: int64

In [ ]:
train.describe(include = 'all')

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
count,11094.000000,11094,11094,11093.000000,11094.000000,9131.00000,9020.000000,8782.000000,11094.000000
unique,NaN,35,2179,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,"[Mico Layout, Stage 2, BTM Layout, Bangalore]",[North Indian],NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,947,850,NaN,NaN,NaN,NaN,NaN,NaN
mean,3736.723274,NaN,NaN,202.708916,53.344511,3.61346,244.544457,123.247893,37.056066
std,2163.307015,NaN,NaN,129.839111,18.551245,0.41434,555.094733,321.025216,12.447496
min,0.000000,NaN,NaN,50.000000,0.000000,2.10000,3.000000,1.000000,10.000000
25%,1856.250000,NaN,NaN,100.000000,50.000000,3.30000,19.000000,7.000000,30.000000
50%,3726.500000,NaN,NaN,200.000000,50.000000,3.60000,63.000000,26.000000,30.000000
75%,5625.750000,NaN,NaN,200.000000,50.000000,3.90000,216.000000,91.000000,45.000000


In [ ]:
train['Restaurant']=le.fit_transform(train['Restaurant'])
train['Minimum_Order']=pd.to_numeric(train['Minimum_Order'].str.replace('₹',' '))
train['Average_Cost']=pd.to_numeric(train['Average_Cost'].str.replace('[^0-9]',''))
train['Rating']=pd.to_numeric(train['Rating'].apply(lambda x : np.nan if x in ['Temporarily Closed','Opening Soon','-','NEW'] else x))
train['Votes']=pd.to_numeric(train['Votes'].apply(lambda x : np.nan if x=='-' else x))
train['Reviews']=pd.to_numeric(train['Reviews'].apply(lambda x : np.nan if x=='-' else x))
train['Delivery_Time']=pd.to_numeric(train['Delivery_Time'].str.replace('[^0-9]',''))

In [ ]:
test['Restaurant']=le.fit_transform(test['Restaurant'])
test['Minimum_Order']=pd.to_numeric(test['Minimum_Order'].str.replace('₹',' '))
test['Average_Cost']=pd.to_numeric(test['Average_Cost'].str.replace('[^0-9]',''))
test['Rating']=pd.to_numeric(test['Rating'].apply(lambda x : np.nan if x in ['Temporarily Closed','Opening Soon','-','NEW'] else x))
test['Votes']=pd.to_numeric(test['Votes'].apply(lambda x : np.nan if x=='-' else x))
test['Reviews']=pd.to_numeric(test['Reviews'].apply(lambda x : np.nan if x=='-' else x))
test['Rating']=test['Rating'].fillna(test['Rating'].median())
test['Votes']=test['Votes'].fillna(test['Votes'].mode()[0])
test['Reviews']=test['Reviews'].fillna(test['Reviews'].median())
test['Average_Cost']=test['Average_Cost'].fillna(test['Average_Cost'].mean())
#sample['Delivery_Time'] = sample['Delivery_Time'].str.replace(' minutes','').astype('int64')

In [ ]:
# train location data one hot encoding
for i in range(len(train)):
    train['Location'][i] = train['Location'][i].split(',')
    train['Location'][i] = [line.strip() for line in train['Location'][i]]

train_location_df = pd.DataFrame()

for n in range(len(train)):
    location = train['Location'][n]
    location_df = pd.DataFrame()
    for i in location:
        location_df[i] = [1]

    train_location_df = pd.concat([train_location_df, location_df], ignore_index=True)

train_location_df.rename(columns ={"" : 'None'}, inplace = True )
train_location_df.head()

,FTI College,Law College Road,Pune,Sector 3,Marathalli,Mumbai Central,Sector 1,Noida,Rmz Centennial,I Gate,...,Nathan Road,Mangaldas Road,Panjetan Colony,Raja Bazar,Jaya Nagar,Saidabad,Noorkhan Bazaar,Musi Nagar,BTM Layout 1,Electronic City
0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# train cuisines data one hot encoding
for i in range(len(train)):
    train['Cuisines'][i] = train['Cuisines'][i].split(',')
    train['Cuisines'][i] = [line.strip() for line in train['Cuisines'][i]]

train_cuisines_df = pd.DataFrame()

for n in range(len(train)):
    cuisines = train['Cuisines'][n]
    cuisines_df = pd.DataFrame()
    for i in cuisines:
        cuisines_df[i] = [1]

    train_cuisines_df = pd.concat([train_cuisines_df, cuisines_df], ignore_index=True)

train_cuisines_df.rename(columns ={"" : 'None'}, inplace = True )
train_cuisines_df.head()

,Fast Food,Rolls,Burger,Salad,Wraps,Ice Cream,Desserts,Italian,Street Food,Mughlai,...,Nepalese,Indian,Tex-Mex,Hot dogs,South American,Middle Eastern,African,Turkish,Israeli,Greek
0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# test location data one hot encoding
for i in range(len(test)):
    test['Location'][i] = test['Location'][i].split(',')
    test['Location'][i] = [line.strip() for line in test['Location'][i]]

test_location_df = pd.DataFrame()

for n in range(len(test)):
    location = test['Location'][n]
    location_df = pd.DataFrame()
    for i in location:
        location_df[i] = [1]

    test_location_df = pd.concat([test_location_df, location_df], ignore_index=True)

test_location_df.rename(columns ={"" : 'None'}, inplace = True )
test_location_df.head()

,Mico Layout,Stage 2,BTM Layout,Bangalore,Sector 1,Noida,Babarpur,New Delhi,Delhi,Yerawada,...,Panjetan Colony,Chatta Bazaar,Jaya Nagar,Saidabad,Gora Bazar,Rajbari,North Dumdum,Hyderabad Public School,Begumpet,Noorkhan Bazaar
0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# test cuisines data one hot encoding

for i in range(len(test)):
    test['Cuisines'][i] = test['Cuisines'][i].split(',')
    test['Cuisines'][i] = [line.strip() for line in test['Cuisines'][i]]
test_cuisines_df = pd.DataFrame()

for n in range(len(test)):
    cuisines = test['Cuisines'][n]
    cuisines_df = pd.DataFrame()
    for i in cuisines:
        cuisines_df[i] = [1]

    test_cuisines_df = pd.concat([test_cuisines_df, cuisines_df], ignore_index=True)

test_cuisines_df.rename(columns ={"" : 'None'}, inplace = True )
test_cuisines_df.head()

,North Indian,Chinese,Assamese,Biryani,Kebab,Fast Food,Mithai,South Indian,Kerala,Seafood,...,Vietnamese,Awadhi,Indian,French,German,Spanish,North Eastern,Chettinad,African,Iranian
0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Fill columns without each other
number = 0
ls=[]
for i in train_cuisines_df.columns:
    ls.append(i)
    ls[number].replace(' ','')
    number = number+1

number = 0
ls2=[]
for i in test_cuisines_df.columns:
    ls2.append(i)
    ls2[number].replace(' ','')
    number = number +1

complement = list(set(ls) - set(ls2))
complement2 = list(set(ls2) - set(ls))

for i in complement2:
    train_cuisines_df[i] = np.nan
train_cuisines_df.rename(columns ={"" : 'None'}, inplace = True )
train_cuisines_df.head()

for i in complement:
    test_cuisines_df[i] = np.nan
test_cuisines_df.rename(columns ={"" : 'None'}, inplace = True )
test_cuisines_df.head()

,North Indian,Chinese,Assamese,Biryani,Kebab,Fast Food,Mithai,South Indian,Kerala,Seafood,...,Bohri,Portuguese,Gujarati,Nepalese,Sri Lankan,Konkan,Indonesian,Tex-Mex,Tamil,Charcoal Chicken
0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Normalize and fill in missing values
train_02=train.copy()
del train_02['Cuisines']
del train_02['Location']
train_02=train_02.drop('Delivery_Time',axis=1)
scaler=MinMaxScaler()
X_scaled_d = scaler.fit_transform(train_02)
X_scaled = pd.DataFrame(X_scaled_d, index=train_02.index, columns=train_02.columns)
X_scaled = X_scaled[['Restaurant']]
train_02 = pd.merge(X_scaled,train_location_df, left_index =True, right_index =True, how= 'left')
train_02 = pd.merge(train_02,train_cuisines_df, left_index =True, right_index =True, how= 'left')
train_02 = train_02.fillna(0)
x=train_02
y=train['Delivery_Time']

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=50)

In [ ]:
model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)
scores = cross_val_score(model_lr, X_train,y_train, cv=10) # model, train, target, cross validation
print('cross-val-score \n{}'.format(scores))
print('cross-val-score.mean \n{:.3f}'.format(scores.mean()))

cross-val-score 
[0.65894466 0.66924067 0.67310167 0.65250965 0.64993565 0.66237113
 0.67139175 0.66237113 0.65335052 0.68170103]
cross-val-score.mean 
0.663


In [ ]:
pred = model_lr.predict(X_test)
print("Test Accuracy: {}%".format(round(model_lr.score(X_test, y_test)*100, 2)))
print(classification_report(y_test, pred))

Test Accuracy: 68.07%
              precision    recall  f1-score   support

          20       0.00      0.00      0.00         3
          30       0.69      0.97      0.81      2265
          45       0.46      0.10      0.16       766
          65       0.14      0.01      0.01       271
          80       0.00      0.00      0.00         5
         120       0.00      0.00      0.00        19

    accuracy                           0.68      3329
   macro avg       0.22      0.18      0.16      3329
weighted avg       0.59      0.68      0.59      3329



In [ ]:
from sklearn.ensemble import RandomForestClassifier
Rforest = RandomForestClassifier()

In [ ]:
Rforest.fit(X_train,y_train)
Rforest.score(X_test,y_test)

0.6233103033944127

In [ ]:
Predictions = Rforest.predict(X_test)
ans=pd.DataFrame(Predictions, columns = ['Delivery_Time'])

In [ ]:
ans.head(100)

,Delivery_Time
0,30
1,30
2,30
3,30
4,30
...,...
95,30
96,45
97,30
98,30
